In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from geopy.distance import great_circle as vc 
import math as Math

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

import math, time
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

import random
import os
import sys

sys.path.insert(0, "/content/sample_data/")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tqdm.notebook import tqdm

class load_data(Dataset):
    def __init__(self, x, y):
        self.length = x.shape[0]
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    def __len__(self):
        return self.length


# A simple utility function for converting pytorch tensors to numpy
def tensor_to_numpy(data, device_type="cpu"):
    if device_type == "cpu":
        return data.detach().numpy()
    else:
        return data.cpu().detach().numpy()


# A simple utility function for converting numpy to pytorch tensors
def numpy_to_tensor(data, dtype="float", device="cuda"):
    if dtype=="float":
        return torch.tensor(data, dtype=torch.float).to(device)
    elif dtype=="long":
        return torch.tensor(data, dtype=torch.long).to(device)





seed = 1024
def reset_random_seeds():
   print("resetting seeds")
   os.environ['PYTHONHASHSEED']=str(seed)
#    tensorflow.random.set_seed(seed)
   np.random.seed(seed)
   random.seed(seed)

reset_random_seeds()


#Supressing Warnings
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

path = '/content/sample_data/' # Path of file
# path = '/content/' # Path of file

filePath = path + 'IMDDataTillLandfall_moreFeatures2.csv'
data = pd.read_csv(filePath, index_col=0)

faniData = data[data['cyclone_id'].str.contains('2019-2')]
print('faniData Shape', faniData.shape)
# bulbulData = data[data['cyclone_id'].str.contains('2019-9')]
# print('bulbulData Shape', bulbulData.shape)
# gajaData = data[data['cyclone_id'].str.contains('2018-13')]
# print('gajaData Shape', gajaData.shape)

#Removing Fani, Bulbul, Gaja data from dataset
data = data[~data['cyclone_id'].str.contains('2019_2')]
# data = data[~data['cyclone_id'].str.contains('2019_9')]
# data = data[~data['cyclone_id'].str.contains('2018_13')]

print('data  Shape', data.shape)


#Normalizing features
scalers = {}
# columnsToNormalize = ['ecp','sst','distance','direction','mssw','lati','long']
# columnsToNormalize = ['ecp','sst','distance','direction','lati','long', 'season', 'pd','diam', 'grade']
# columnsToNormalize = ['diam','distance', 'direction', 'lati', 'long']

# 24 steps GC
# columnsToNormalize = ['cino', 'mssw', 'pd', 'distance', 'long']

# mutual information
columnsToNormalize = ['lati', 'cino', 'ecp', 'mssw', 'pd', 'sst', 'distance', 'direction', 'long']



for col in columnsToNormalize:
  scaler = StandardScaler()
  data[[col]] = scaler.fit_transform(data[[col]])
  scalers[col] =  scaler

data_pad = []

for key in np.unique(data['cyclone_id']):
    data_pad += [data[ data.loc[:, 'cyclone_id'] == key].loc[:, ['lati', 'cino', 'ecp', 'mssw', 'pd', 'sst', 'distance', 'direction', 'long']].values]

#Normalizing the features of Fani, Gaja, Bulbul with same scalers
for col in columnsToNormalize:
  scaler = scalers[col]
  faniData[[col]] = scaler.transform(faniData[[col]])

data_padF = []
data_padF += [faniData.loc[:, ['lati', 'cino', 'ecp', 'mssw', 'pd', 'sst', 'distance', 'direction', 'long']].values]

# for col in columnsToNormalize:
#   scaler = scalers[col]
#   bulbulData[[col]] = scaler.transform(bulbulData[[col]])

# data_padB = []
# data_padB += [bulbulData.loc[:, ['cino', 'ecp', 'mssw', 'pd', 'distance', 'direction', 'lati', 'long']].values]


# for col in columnsToNormalize:
#   scaler = scalers[col]
#   gajaData[[col]] = scaler.transform(gajaData[[col]])

# data_padG = []
# data_padG += [gajaData.loc[:, ['cino', 'ecp', 'mssw', 'pd', 'distance', 'direction', 'lati', 'long']].values]


#Preapring training dataset
def load_dataLong(data_pad, trainPoints ,amount_of_features):
    X = []
    y = []
    for cdata in data_pad:
      length = len(cdata)
      diff = length - trainPoints
      if diff > 0:
        yy = cdata[length-1][-1]
        for i in range(diff):
          cc = cdata[i : i + trainPoints]
          X.append(cc)
          y.append(yy)
    X = np.array(X)
    y = np.array(y)
    x_train = np.reshape(X, (X.shape[0], X.shape[1], amount_of_features)) 
    #y_train = np.reshape(y, (y.shape[0], y.shape[1], 1))
    return [x_train, y]

def build_model(feature_count, loader_train, device, epochs):
    model = HSM1(device, feature_count, 64)
    model = model.to(device)
    print(model.parameters)
    criterion = nn.MSELoss()
    # criterion = nn.SmoothL1Loss()
    # Define optimizer
    optimizer = optim.Adamax(model.parameters(), lr=0.001)
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


    for epoch in tqdm(range(epochs)):
        # print("\n\nEpoch: ", epoch+1, "\n-----------------")
        running_loss = 0
        
        for index, data in enumerate(loader_train):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            #print(inputs.shape)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs).squeeze()
            # Compute loss for each timestep
            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()
            running_loss+= loss.item()
        if (epoch+1)%10 == 0:
            print("Training loss: ", running_loss/len(loader_train))
    return model


def get_pred(model, device, loader):
    actual, preds = list(), list()
    # Make sure no gradients are being used during evaluation
    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            labels = tensor_to_numpy(labels, device)
            pred = model(inputs).squeeze()
            pred = tensor_to_numpy(pred, device)
            actual.extend(labels)
            preds.extend(pred)
    return actual, preds

import torch.nn.functional as F

# relu
def new(x):
  relu1 = nn.ReLU(inplace=False)
  return relu1(x)

# # gelu
# def new(x):
#   gelu1 = nn.GELU()
#   return gelu1(x)


# # elu
# def new(x):
#   elu1 = nn.ELU()
#   return elu1(x)

# # mish activation
# def new(x):
#     return 1.5*x* (torch.tanh(F.softplus(x)))
#     #return x/(1+tf.exp(-2*x))

# swish activation
# def new(x):
#     return 0.5*x*(1 + torch.tanh(x))
#     #return x/(1+tf.exp(-2*x))


feature_count = 9
trains = [4, 6, 8, 12]
# trains = [12]

resetting seeds
faniData Shape (58, 14)
data  Shape (3485, 14)


In [ ]:
# DILATED RNN

In [2]:
class HSM1_layer(nn.Module):
    def __init__(self, dilation, input_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=None, return_sequences=True):
        # Define all the variables required
        super(HSM1_layer, self).__init__()
        self.dilation = dilation
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.use_gru = use_gru
        self.use_lstm = use_lstm
        self.use_rnn = use_rnn
        self.device = device
        self.return_sequences = return_sequences

        if self.use_gru == True:
            self.rnn = nn.GRU(self.input_dim, self.hidden_dim, batch_first=True)
        elif self.use_lstm == True:
            self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, batch_first=True)
        elif self.use_rnn == True:
            self.rnn = nn.RNN(self.input_dim, self.hidden_dim, batch_first=True)

    def forward(self, data):
        final_out = self._cell_output(data, self.rnn)
        if not self.return_sequences:
            return final_out[:,-1,:]
        else:
            return final_out

    def _cell_output(self, data, cell):
        output = self.drnn_layer(cell, data)
        return output
    
    def drnn_layer(self, cell, data):
        batch_size = data.shape[0]
        n_steps = data.shape[1]
        hidden_size = cell.hidden_size

        #print("\n======================= DILATION:", self.dilation, "=====================================")

        data = self._pad_inputs(data, batch_size, n_steps)
        #print("After Padding:", data.shape)
        data = self._prepare_inputs(data)
        data = self._apply_cell(data, cell)
        #print("After Applying cell:", data.shape)
        data = self._split_outputs(data, batch_size, hidden_size)
        #print("After Splitting outputs:", data.shape)
        outputs = self._unpad_outputs(data, n_steps)
        #print("After Unpadding:", outputs.shape)
        #print("\n")

        return outputs
    
    def _apply_cell(self, data, cell):
        data, _ = cell(data)
        return data

    def _unpad_outputs(self, data, n_steps):
        return data[:,:n_steps,:]

    def _split_outputs(self, data, batch_size, hidden_size):
        n_steps = data.shape[1]*self.dilation
        numerator = data.shape[1]
        new_data = torch.reshape(data, (batch_size, n_steps, hidden_size))
        new_data = torch.cat([new_data[:, i::numerator, :] for i in range(numerator)], dim=1)
        return new_data

    def _pad_inputs(self, data, batch_size, n_steps):
        is_even = (n_steps % self.dilation) == 0

        if not is_even:
            dilated_steps = n_steps // self.dilation + 1
            zeros_ = torch.zeros(batch_size, dilated_steps * self.dilation - data.shape[1], data.shape[2]).to(self.device)
            data = torch.cat((data, zeros_), dim=1)
        else:
            dilated_steps = n_steps // self.dilation

        return data

    def _prepare_inputs(self, data):
        batch_size = data.shape[0]
        new_data = torch.cat([data[:, i::self.dilation, :] for i in range(self.dilation)], dim=1)
        
        return torch.reshape(new_data, (self.dilation*batch_size, data.shape[1]//self.dilation, data.shape[2]))

In [ ]:
# Initialize Model

In [3]:
# Create an HSM1 model
class HSM1(nn.Module):
    def __init__(self, device, input_dim, hidden_dim):
        super(HSM1, self).__init__()
        self.layer1 = HSM1_layer(1, input_dim, 1024,  use_gru=True, use_lstm=False, use_rnn=False, device=device, return_sequences=True)
        self.layer2 = HSM1_layer(2, 1024, 512,  use_gru=True, use_lstm=False, use_rnn=False, device=device, return_sequences=True)
        self.layer3 = HSM1_layer(4, 512, 256, use_gru=True, use_lstm=False, use_rnn=False, device=device, return_sequences=False)
        # self.layer4 = HSM1_layer(8, 128, 64, use_gru=False, use_lstm=True, use_rnn=False, device=device, return_sequences=False)
        # self.layer4 = HSM1_layer(8, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer5 = HSM1_layer(16, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer6 = HSM1_layer(32, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer7 = HSM1_layer(64, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer8 = HSM1_layer(128, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=True)
        # self.layer9 = HSM1_layer(256, hidden_dim, hidden_dim, use_gru=False, use_lstm=False, use_rnn=True, device=device, return_sequences=False)
        self.linear = nn.Linear(256, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = new(x)
        x = self.layer2(x)
        x = new(x)
        x = self.layer3(x)
        x = new(x)
        # x = self.layer4(x)
        # x = new(x)
        # x = self.layer5(x)
        # x = self.layer6(x)
        # x = self.layer7(x)
        # x = self.layer8(x)
        # x = self.layer9(x)
        x = self.linear(x)
        return x

In [ ]:
# rest of the code

In [4]:
def getMaeRmse(predictions, y_test, scalers):
  length = len(predictions)
#   latiScaler = scalers['lati']
  longScaler = scalers['long']
  #Extracting predicted and acutal latis
#   latisP = predictions
  longsP = predictions
#   latisA = y_test
  longsA = y_test

#   latisA = np.reshape(latisA,(length,1))
  longsA = np.reshape(longsA,(length,1))
#   latisP = np.reshape(latisP,(length,1))
  longsP = np.reshape(longsP,(length,1))

  #Getting actual values of latitiude/longitude
#   latisA = latiScaler.inverse_transform(latisA)
  longsA = longScaler.inverse_transform(longsA)
#   latisP = latiScaler.inverse_transform(latisP)
  longsP = longScaler.inverse_transform(longsP)

#   testScorela = mean_squared_error(latisA, latisP)
  testScorelo = mean_squared_error(longsA, longsP)

#   testScorelaMae = mean_absolute_error(latisA, latisP)
  testScoreloMae = mean_absolute_error(longsA, longsP)

#   return np.sqrt(testScorela), np.sqrt(testScorelo), testScorelaMae, testScoreloMae
  return np.sqrt(testScorelo), testScoreloMae



In [ ]:
# def getDistance(predictions, y_test, scalers):
#   length = len(predictions)
#   latiScaler = scalers['lati']
#   longScaler = scalers['long']
#   latisP = predictions[:,0]
#   longsP = predictions[:,1]
#   latisA = y_test[:,0]
#   longsA = y_test[:,1]

#   latisA = np.reshape(latisA,(length,1))
#   longsA = np.reshape(longsA,(length,1))
#   latisP = np.reshape(latisP,(length,1))
#   longsP = np.reshape(longsP,(length,1))

#   latisA = latiScaler.inverse_transform(latisA)
#   longsA = longScaler.inverse_transform(longsA)
#   latisP = latiScaler.inverse_transform(latisP)
#   longsP = longScaler.inverse_transform(longsP)
#   dist = 0
#   distM = 0
#   for i in range(length):
#     actualCor = [latisA[i][0], longsA[i][0]]
#     predictedCor = [latisP[i][0], longsP[i][0]]
#     dist = dist + vc(actualCor, predictedCor).kilometers
#     distM = distM + vc(actualCor, predictedCor).miles

#   return dist/length, distM/length


In [5]:
for i in range(len(trains)):
  trainPoints = trains[i]
  print("===========================================================")
  print("T (size of training point) =   ", trainPoints)
  X, y = load_dataLong(data_pad, trainPoints, feature_count)
  XF, yF = load_dataLong(data_padF, trainPoints, feature_count)
#   XG, yG = load_dataLong(data_padG, trainPoints, feature_count)
#   XB, yB = load_dataLong(data_padB, trainPoints, feature_count)

  datasetF = load_data(numpy_to_tensor(XF, "float", device), numpy_to_tensor(yF, "float", device))
  

  loaderF = DataLoader(dataset=datasetF, batch_size=256, shuffle=False, num_workers=0)


  X, y = shuffle(X, y, random_state=seed)

  print("No of train points", X.shape)

  kf = KFold(n_splits=5, shuffle=True,  random_state=seed)

  scoreslo = []
  scoresFlo = []
#   scoresGla = []
#   scoresBla = []

  scoresloMae = []
  scoresFloMae = []
#   scoresGlaMae = []
#   scoresBlaMae = []
    
#   scoreslo = []
#   scoresFlo = []
#   scoresGlo = []
#   scoresBlo = []

#   scoresloMae = []
#   scoresFloMae = []
#   scoresGloMae = []
#   scoresBloMae = []

#   scoresMi = []
#   scoresFMi = []
#   scoresGMi = []
#   scoresBMi = []

#   scoresKm = []
#   scoresFKm = []
#   scoresGKm = []
#   scoresBKm = []

  for train_index, test_index in kf.split(X):
      X_train = X[train_index]
      y_train = y[train_index]
      X_test = X[test_index]
      y_test = y[test_index]
        
      dataset_train = load_data(numpy_to_tensor(X_train, "float", device), numpy_to_tensor(y_train, "float", device))
      dataset_test = load_data(numpy_to_tensor(X_test, "float", device), numpy_to_tensor(y_test, "float", device))

      loader_train = DataLoader(dataset=dataset_train, batch_size=64, shuffle=True, num_workers=0)
      loader_test = DataLoader(dataset=dataset_test, batch_size=512, shuffle=False, num_workers=0)

    

      model = build_model(feature_count, loader_train, device, epochs = 250)
      
      
        
      y_test, predictions = get_pred(model, device, loader_test)
      yF, predictionsF = get_pred(model, device, loaderF)
          
      
    
#       predictions = model.predict(X_test)
#       predictionsF = model.predict(XF)
#       predictionsG = model.predict(XG)
#       predictionsB = model.predict(XB)

      rmselo, mselo = getMaeRmse(predictions, y_test, scalers)
      rmseloF, mseloF = getMaeRmse(predictionsF, yF, scalers)
#       rmselaG, mselaG = getMaeRmse(predictionsG, yG, scalers)
#       rmselaB, mselaB = getMaeRmse(predictionsB, yB, scalers)

#       distKm, distMi = getDistance(predictions, y_test, scalers)
#       distKmF, distMiF = getDistance(predictionsF, yF, scalers)
#       distKmG, distMiG = getDistance(predictionsG, yG, scalers)
#       distKmB, distMiB = getDistance(predictionsB, yB, scalers)

#       scoresMi.append(distMi)
#       scoresFMi.append(distMiF)
#       scoresGMi.append(distMiG)
#       scoresBMi.append(distMiB)

#       scoresKm.append(distKm)
#       scoresFKm.append(distKmF)
#       scoresGKm.append(distKmG)
#       scoresBKm.append(distKmB)

      scoreslo.append(rmselo)
      scoresFlo.append(rmseloF)
#       scoresGla.append(rmselaG)
#       scoresBla.append(rmselaB)
    
#       scoreslo.append(rmselo)
#       scoresFlo.append(rmseloF)
#       scoresGlo.append(rmseloG)
#       scoresBlo.append(rmseloB)

      scoresloMae.append(mselo)
      scoresFloMae.append(mseloF)
#       scoresGlaMae.append(mselaG)
#       scoresBlaMae.append(mselaB)
    
#       scoresloMae.append(mselo)
#       scoresFloMae.append(mseloF)
#       scoresGloMae.append(mseloG)
#       scoresBloMae.append(mseloB)

      print('scores longitude  RMSE ', scoreslo)
#       print('scores longitude  RMSE', scoreslo)
    
      print('scores longitude Mae ', scoresloMae)
#       print('scores longitude  Mae', scoresloMae)

#       print('distance Miles  ', scoresMi)
#       print('distances KiloMeters  ', scoresKm)

  print("5 fold accuracy Long RMSE ",np.array(scoreslo).mean())
  print("5 fold accuracy Long Fani RMSE ", np.array(scoresFlo).mean())
#   print("5 fold accuracy Lati Gaja RMSE ", np.array(scoresGla).mean())
#   print("5 fold accuracy Lati Bulbul RMSE ", np.array(scoresBla).mean())
    
#   print("5 fold accuracy Long RMSE ",np.array(scoreslo).mean())
#   print("5 fold accuracy Long Fani RMSE ", np.array(scoresFlo).mean())
#   print("5 fold accuracy Long Gaja RMSE ", np.array(scoresGlo).mean())
#   print("5 fold accuracy Long Bulbul RMSE ", np.array(scoresBlo).mean())

  print("5 fold accuracy  Long MAE",np.array(scoresloMae).mean())
  print("5 fold accuracy  Long  Fani MAE", np.array(scoresFloMae).mean())
#   print("5 fold accuracy  Lati Gaja  MAE", np.array(scoresGlaMae).mean())
#   print("5 fold accuracy  Lati Bulbul MAE", np.array(scoresBlaMae).mean())
    
#   print("5 fold accuracy Long MAE",np.array(scoresloMae).mean())
#   print("5 fold accuracy  Long  Fani MAE", np.array(scoresFloMae).mean())
#   print("5 fold accuracy  Lati  Gaja MAE", np.array(scoresGloMae).mean())
#   print("5 fold accuracy  Lati  Bulbul MAE", np.array(scoresBloMae).mean())

  # print("5 fold distance average Miles ",np.array(scoresMi).mean())
  # print("Fani average error Miles ", np.array(scoresFMi).mean())
  # print("Gaja average error Miles ", np.array(scoresGMi).mean())
  # print("Bulbul average error Miles ", np.array(scoresBMi).mean())

#   print("5 fold distance average Kilometers ",np.array(scoresKm).mean())
#   print("5 fold distance average Fani Kilometers ", np.array(scoresFKm).mean())
#   print("5 fold distance average Gaja Kilometers ", np.array(scoresGKm).mean())
#   print("5 fold distance average Bulbul Kilometers ", np.array(scoresBKm).mean())


T (size of training point) =    4
No of train points (2820, 4, 9)
<bound method Module.parameters of HSM1(
  (layer1): HSM1_layer(
    (rnn): GRU(9, 1024, batch_first=True)
  )
  (layer2): HSM1_layer(
    (rnn): GRU(1024, 512, batch_first=True)
  )
  (layer3): HSM1_layer(
    (rnn): GRU(512, 256, batch_first=True)
  )
  (linear): Linear(in_features=256, out_features=1, bias=True)
)>


  0%|          | 0/250 [00:00<?, ?it/s]

Training loss:  0.10455837556057507
Training loss:  0.05242394454156359
Training loss:  0.026935391335023776
Training loss:  0.015242783585563302
Training loss:  0.011239893048898213
Training loss:  0.004846470035974764
Training loss:  0.004928942491662585
Training loss:  0.005305332118748791
Training loss:  0.0011916473465842297
Training loss:  0.0011896439908822584
Training loss:  0.0009342644366875498
Training loss:  0.0020325326428671056
Training loss:  0.0006346854190471479
Training loss:  0.0006840125958357627
Training loss:  0.00030717335701208666
Training loss:  0.002613076377504816
Training loss:  0.0001327377297760298
Training loss:  0.00014858638739193944
Training loss:  0.00034888458281380334
Training loss:  0.00012052969779890393
Training loss:  0.00046192464853649854
Training loss:  0.00018272678203680294
Training loss:  0.0002460934900025475
Training loss:  0.0002203052123756303
Training loss:  5.917211425791417e-05
scores longitude  RMSE  [0.96844476]
scores longitude M

  0%|          | 0/250 [00:00<?, ?it/s]

Training loss:  0.09543968437032567
Training loss:  0.04999963535616795
Training loss:  0.02269923174753785
Training loss:  0.015828395133010216
Training loss:  0.008572513793802096
Training loss:  0.008334562350581918
Training loss:  0.005848751668559594
Training loss:  0.0034078356644992405
Training loss:  0.002138616184108994
Training loss:  0.002635024568816233
Training loss:  0.0011187551960271473
Training loss:  0.0011801559869329343
Training loss:  0.001517350971375385
Training loss:  0.0005439098434483943
Training loss:  0.0009649586669790248
Training loss:  0.0007823183865790876
Training loss:  0.0007982209111408641
Training loss:  0.0004253904339242985
Training loss:  0.0009621663630241528
Training loss:  0.00015633966924926627
Training loss:  0.0003943374249502085
Training loss:  7.480279034805587e-05
Training loss:  8.617329290751109e-05
Training loss:  0.0014029467727393946
Training loss:  5.271869248544034e-05
scores longitude  RMSE  [0.96844476, 1.4127339]
scores longitu

...

Training loss:  0.07363858011861642
Training loss:  0.0341155258162568
Training loss:  0.012774023537834486
Training loss:  0.008968115915195085
Training loss:  0.005402889738131004
Training loss:  0.0025190554976385706
Training loss:  0.001607688573130872
Training loss:  0.0017069080980339397
Training loss:  0.0011757767279050313
Training loss:  0.0038596220983890817
Training loss:  0.001050411721128815
Training loss:  0.0008335372658621054
Training loss:  0.0008329262952126252
Training loss:  0.0017750052405366052
Training loss:  0.000466154361978018
Training loss:  0.00041349723445212777
Training loss:  0.000341531288237699
Training loss:  0.00049630522746232
Training loss:  0.0008902262191744134
Training loss:  0.0005135930380978001
Training loss:  0.0004085614655195968
Training loss:  0.00043528281230464927
Training loss:  0.00042743697864959057
Training loss:  0.00014945702848005263
Training loss:  0.00023915271231089719
scores longitude  RMSE  [0.32110494, 0.45588714, 0.47085828

  0%|          | 0/250 [00:00<?, ?it/s]

Training loss:  0.07151542728145917
Training loss:  0.032213210981960096
Training loss:  0.010634355266423276
Training loss:  0.00599423191548946
Training loss:  0.00579451034233595
Training loss:  0.003067742358931961
Training loss:  0.0032058041009198255
Training loss:  0.003631317561181883
Training loss:  0.0009604472309850584
Training loss:  0.002674108065548353
Training loss:  0.0013735588096703093
Training loss:  0.001072308636745826
Training loss:  0.0005857903173212738
Training loss:  0.0018291565453788887
Training loss:  0.0004034089500540479
Training loss:  0.0003016238812657927
Training loss:  0.0003256338804931147
Training loss:  0.0016959507314216655
Training loss:  0.0005420783115065811
Training loss:  0.00011266468535116776
Training loss:  0.00015697194612585008
Training loss:  0.0013248848481453024
Training loss:  0.0004430507979122922
Training loss:  7.367175733937377e-05
Training loss:  9.353899379978732e-05
scores longitude  RMSE  [0.32110494, 0.45588714, 0.47085828,

  0%|          | 0/250 [00:00<?, ?it/s]

Training loss:  0.06617904574765514
Training loss:  0.029655655147507787
Training loss:  0.012715148351465663
Training loss:  0.008063664679260304
Training loss:  0.004962560914767285
Training loss:  0.0036458273437650255
Training loss:  0.0012123164112078182
Training loss:  0.0036970119496497014
Training loss:  0.000756857639013712
Training loss:  0.0005880245262233075
Training loss:  0.0007826097365371728
Training loss:  0.00046560236827038654
Training loss:  0.0009624370541132521
Training loss:  0.004394971047683309
Training loss:  0.0008958451726357453
Training loss:  0.0005442767754478458
Training loss:  0.0004401060868985951
Training loss:  0.0010611703440493632
Training loss:  0.0001700717223987643
Training loss:  0.0008070247119273214
Training loss:  0.002168432909911644
Training loss:  0.00015515282060126387
Training loss:  0.0001170926201060259
Training loss:  0.0012754102378191117
Training loss:  0.00013527797424709812
scores longitude  RMSE  [0.32110494, 0.45588714, 0.47085

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Generateing latitiude/longitude grpahs for Fani, Gaja and Bulbul
trainPoints = 8  # Graphs for T = 8
X, y = load_dataLong(data_pad, trainPoints, feature_count)
XF, yF = load_dataLong(data_padF, trainPoints, feature_count)
# XG, yG = load_dataLong(data_padG, trainPoints, feature_count)
# XB, yB = load_dataLong(data_padB, trainPoints, feature_count)
X, y = shuffle(X, y, random_state=seed)
model = build_model(feature_count, trainPoints)
model.fit(X, y, batch_size=512, epochs=150, verbose=0, shuffle = True)
predictionsF = model.predict(XF)
predictionsG = model.predict(XG)
predictionsB = model.predict(XB)

In [ ]:
# For Fani cyclone
print("trainPoints  (T) = ", trainPoints)
latiScaler = scalers['lati']
# longScaler = scalers['long']

lengthF = len(yF)

latisPF = predictionsF
# longsPF = predictionsF[:,1]
latisAF = yF
# longsAF = yF[:,1]


latisPF = np.reshape(latisPF,(lengthF,1))
# longsPF = np.reshape(longsPF,(lengthF,1))
latisAF = np.reshape(latisAF,(lengthF,1))
# longsAF = np.reshape(longsAF,(lengthF,1))

latisPF = latiScaler.inverse_transform(latisPF)
# longsPF = longScaler.inverse_transform(longsPF)
latisAF = latiScaler.inverse_transform(latisAF)
# longsAF = longScaler.inverse_transform(longsAF)

plt.figure(figsize=(10, 10))
fig, ax1 = plt.subplots(figsize=(12, 4))
a = np.linspace(1,lengthF,lengthF)*3 + 8*3
color = 'tab:red'
ax1.set_xlabel('Hour (during cyclone progress)')
ax1.set_ylabel('Latitude', color=color)
ax1.plot(a, latisAF, marker = '*',label = "Lati at Landfall",color = 'red')
ax1.plot(a, latisPF, marker = '+',label = "Predicted Lati at Landfall",color ='green')
ax1.tick_params(axis='y', labelcolor=color)
ax1.xaxis.set_ticks(a)
plt.xticks(rotation=60)
plt.ylim((19.25,20.75))
ax1.legend(loc=1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Longitude', color=color)  # we already handled the x-label with ax1
ax2.plot(a, longsAF, marker = '*',label = "Long at Landfall",color = 'blue')
ax2.plot(a, longsPF, marker = '+',label = "Predicted Long at Landfall",color='magenta')

ax2.tick_params(axis='y', labelcolor=color)
plt.ylim((85.5,87))
fig.tight_layout()  # otherwise the right y-label is slightly clipped
ax2.legend(loc=2)
plt.show()

In [ ]:
#Time to calculate 150 epochs
X, y = load_dataLong(data_pad, 4, feature_count)
X, y = shuffle(X, y, random_state=0)
model = build_model(feature_count, 4)
start = time.time()
model.fit(X, y, batch_size=512, epochs=150, verbose=0, shuffle = True)
end = time.time()
print("time (seconds) ", end-start)

In [ ]:
#Plotting the Model structure
model = build_model(feature_count, 4)
X, y = load_dataLong(data_pad, 4, feature_count)
model.fit(X, y, batch_size=512, epochs=10, verbose=0, shuffle = True)
plot_model(model, to_file='model2.png', show_shapes=True, show_layer_names=False)